In [1]:
# | default_exp paths
# %reload_ext nb_black
%reload_ext autoreload
%autoreload 2

In [2]:
# | export
# | hide
from fastcore.basics import patch_to
from fastcore.foundation import L
from pathlib import Path
from functools import wraps
from torch_snippets.loader import (
    choose as ts_choose,
    Tqdm,
    os,
    logger,
    Info,
    Debug,
    Warn,
    Excep,
    common,
)
from functools import lru_cache
import hashlib, shutil
from collections import defaultdict
import glob, json
import dill, time
import subprocess
import shutil

In [3]:
# | export
# | hide
def input_to_str(func):
    @wraps(func)
    def inner(input, *args, **kwargs):
        if isinstance(input, P):
            input = str(input)
        if isinstance(input, list):
            input = [str(i) for i in input]
        out = func(input, *args, **kwargs)
        return out

    return inner


def output_to_path(func):
    @wraps(func)
    def inner(input, *args, **kwargs):
        out = func(input, *args, **kwargs)
        if isinstance(out, str):
            out = P(out)
        if isinstance(out, list):
            if len(out) > 0 and isinstance(out[0], str):
                out = [P(o) for o in out]
        return out

    return inner

In [4]:
# | export
# | hide
def process_f(f):
    f = f.replace("-", "_").replace(".", "__")
    if f[0].isdigit():
        f = "_" + f

    return f


@lru_cache()
def get_fs(fldr):
    fldr = P(fldr)
    __fs = {f: stem(f) for f in fldr.ls()}
    _fs = defaultdict(list)
    for f, sf in __fs.items():
        _fs[sf].append(f)
    fs = {}
    for f, sfs in _fs.items():
        f = process_f(f)
        if len(sfs) == 1:
            fs[f] = sfs[0]
        else:
            for _f in sfs:
                fs[f"{f}__{_f.extn()}"] = _f
    return fs


valid_methods = dir(Path) + ["isfile"]


class P0(Path):
    try:
        from pathlib import _PosixFlavour, _WindowsFlavour

        _flavour = _PosixFlavour() if os.name == "posix" else _WindowsFlavour()
    except ImportError:
        pass

    def __new__(cls, *pathsegments):
        o = super().__new__(cls, *pathsegments)
        return o

    def __init__(self, *args, **kwargs):
        return super().__init__()

    @property
    @lru_cache()
    def isfile(self):
        return self.is_file()

    def ls(self):
        return L(self.iterdir())

    def extn(self, pattern="*"):
        return self.suffix.replace(".", "")

    def __getattr__(self, name):
        if name in valid_methods or self.isfile or self.exists():
            raise AttributeError(
                f"'{self.__class__.__name__}' object has no attribute '{name}'"
            )

        fs = get_fs(self)
        if name in fs:
            return fs[name]
        else:
            raise AttributeError(
                f"'{self.__class__.__name__}' object has no attribute '{name}'"
            )


P = Path


@patch_to(P)
def Glob(self, pattern="*"):
    return L(self.glob(pattern))


def stem(fpath):
    return P(fpath).stem


@input_to_str
def stems(folder, silent=False):
    if isinstance(folder, (str, P)):
        return L([stem(str(x)) for x in Glob(folder, silent=silent)])
    if isinstance(folder, list):
        return L([stem(x) for x in folder])


P.stems = lambda self: stems(self.ls())
ls = lambda self: L(self.iterdir())
P.__repr__ = lambda self: f"» {self}"
P.__og_dir__ = P.__dir__


@patch_to(P)
def __dir__(self):
    if self.isfile:
        return self.__og_dir__()
    fs = get_fs(self)
    return self.__og_dir__() + list(fs.keys())

In [5]:
print(P().ls())
print(P().resolve())

[» _quarto.yml, » decorators.ipynb, » markups.ipynb, » sidebar.yml, » AttrDict.ipynb, » interactive_show.ipynb, » load_defautls.ipynb, » sklegos.ipynb, » bounding_boxes.ipynb, » show.ipynb, » pdf.ipynb, » charts.ipynb, » paths.ipynb, » nbdev.yml, » tmp.csv, » jupyter_notebook.ipynb, » config.ipynb, » misc.ipynb, » registry.ipynb, » adapters.ipynb, » report.ipynb, » .ipynb_checkpoints, » capsule.ipynb, » logging.ipynb, » inspector.ipynb, » bokeh_plotting.ipynb, » index.ipynb, » imgaug_loader.ipynb]
/Users/yeshwanth/Code/Personal/torch_snippets/nbs


In [6]:
# !touch tmp.txt tmp.csv
# # x = P()
# x.tmp__csv, x.tmp__txt, x.misc

In [7]:
!rm tmp.txt tmp.misc

rm: tmp.txt: No such file or directory
rm: tmp.misc: No such file or directory


In [8]:
# | export
@patch_to(P)
def rmtree(self, prompt="Really remove `{self}` and its contents? [y/n] ", force=False):
    if force:
        shutil.rmtree(self)
    elif prompt and input(prompt.format(self=self)).lower() == "y":
        shutil.rmtree(self)
    else:
        raise OSError(f"{self} exists and is not empty")


@patch_to(P)
def size(self):
    if self.is_dir():
        raise Exception(f"`{self}` is a directory")
    fsize = os.path.getsize(self) >> 20
    fsize = f"{fsize} MB" if fsize > 0 else f"{os.path.getsize(self) >> 10} KB"
    return fsize


@patch_to(P, as_prop=True)
def sz(self):
    return self.size()


@patch_to(P, as_prop=True)
def extn(self):
    return self.suffix.replace(".", "")


def extn(fpath):
    return P(fpath).extn


@patch_to(P)
def sample(self, pattern="*"):
    return ts_choose(self.Glob(pattern))


@patch_to(P)
def mv(self, to):
    os.rename(self, to)
    return P(to)


@patch_to(P)
def cp(self, to):
    return P(shutil.copy(self, to))


def remove_file(self, dry_run):
    if dry_run:
        print(f"DRY RUN: Removing {self}")
    else:
        os.remove(self)


@patch_to(P)
def rm(
    self,
    confirm_prompt="Are you sure you want to delete `{self}`? [y/N]",
    silent=True,
    missing_ok=True,
    force=False,
    dry_run=False,
):
    confirm = (
        input(confirm_prompt.format(self=self))
        if (confirm_prompt and not force)
        else "y"
    )
    if confirm.lower() == "y":
        if missing_ok:
            try:
                remove_file(self, dry_run)
            except:
                ...
        else:
            remove_file(self, dry_run)
        if not silent:
            logger.info(f"Deleted {self}")
    else:
        if not silent:
            logger.info(f"Aborting delete: {self}")

In [9]:
p = P("test.txt")
p.touch()
logger.info(p.size())

[08/24/24 11:12:12] INFO     0 KB                                                                                                                   ]8;id=830137;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/1417366103.py:3\1417366103.py]8;;\:]8;id=397950;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/1417366103.py:3#<module>:3\<module>:3]8;;\

Path objects can be moved and copied

In [10]:
p = p.mv("test1.txt")
q = p.cp("test2.txt")

Path objects have a `size`, `extn` (extension) and `parent` attributes

In [11]:
assert isinstance(q, P)
assert q.size() == "0 KB"
assert str(p) == "test1.txt"
assert p.extn == "txt"
assert p.parent == P()

They can be deleted with/without a prompt

In [12]:
p.rm(force=True)
q.rm(confirm_prompt=False)

Folders can be globbed with a default of everything

In [13]:
p = P("../torch_snippets")
assert P().ls() == P().Glob()
p.Glob("*.py")

(#34) [» ../torch_snippets/misc.py,» ../torch_snippets/load_defaults.py,» ../torch_snippets/text_utils.py,» ../torch_snippets/_nbdev.py,» ../torch_snippets/paths.py,» ../torch_snippets/charts.py,» ../torch_snippets/pdf_loader.py,» ../torch_snippets/interactive_show.py,» ../torch_snippets/registry.py,» ../torch_snippets/markup2.py,» ../torch_snippets/_modidx.py,» ../torch_snippets/inspector.py,» ../torch_snippets/__init__.py,» ../torch_snippets/tmp.py,» ../torch_snippets/torch_loader.py,» ../torch_snippets/logger.py,» ../torch_snippets/markup.py,» ../torch_snippets/fastcores.py,» ../torch_snippets/sklegos.py,» ../torch_snippets/cli.py...]

You can sample a random file from the directory

In [14]:
q = p.sample("*.py")
Info(f"Sample file: `{q}`")
Info(f"Sample file size: `{q.size()}`")

                    INFO     Sample file: `../torch_snippets/logger.py`                                                                             ]8;id=885139;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/1725593160.py:2\1725593160.py]8;;\:]8;id=225317;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/1725593160.py:2#<module>:2\<module>:2]8;;\

                    INFO     Sample file size: `9 KB`                                                                                               ]8;id=350834;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/1725593160.py:3\1725593160.py]8;;\:]8;id=367180;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/1725593160.py:3#<module>:3\<module>:3]8;;\

In [15]:
try:
    p.size()
except Exception as e:
    logger.warning(e)

                    WARNING  `../torch_snippets` is a directory                                                                                     ]8;id=875372;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/1980994904.py:4\1980994904.py]8;;\:]8;id=635052;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/1980994904.py:4#<module>:4\<module>:4]8;;\

In [ ]:
# | export
# | hide
def isdir(fpath):
    return os.path.isdir(fpath)


def current_file_dir():
    """This is not current working directory but the directory of the file where this function is called"""
    return P(__file__).parent


@input_to_str
def makedir(x, prompt=False, silent=True):
    create = input(f"Creating folder {x}. Are you sure? [y/n]") if prompt else "y"
    if create:
        os.makedirs(x, exist_ok=True)
    if not silent:
        Info(f"Created folder {x}")


@input_to_str
def fname(fpath):
    return fpath.split("/")[-1]


@input_to_str
def fname2(fpath):
    return stem(fpath.split("/")[-1])


@input_to_str
@output_to_path
def parent(fpath):
    out = "/".join(fpath.split("/")[:-1])
    if out == "":
        return "./"
    else:
        return out


@input_to_str
@output_to_path
def Glob(x, extns=None, silent=False):
    files = glob.glob(x + "/*") if "*" not in x else glob.glob(x)
    if extns:
        if isinstance(extns, str):
            extns = extns.split(",")
        files = [f for f in files if any([f.endswith(ext) for ext in extns])]

    # if not silent: logger.opt(depth=1).log('INFO', '{} files found at {}'.format(len(files), x))
    return files


def find(
    item=None, List=None, match_stem=False, condition=None, return_indexes_also=False
):
    """Find an `item` in a `List`
    >>> find('abc', ['ijk','asdfs','dfsabcdsf','lmnop'])
    'dgsabcdsf'
    >>> find('file1', ['/tmp/file0.jpg', '/tmp/file0.png', '/tmp/file1.jpg', '/tmp/file1.png', '/tmp/file2.jpg', '/tmp/file2.png'])
    ['/tmp/file1.jpg', '/tmp/file1.png']
    """
    if callable(condition):
        filtered = [(ix, i) for ix, i in enumerate(List) if condition(i)]

    else:
        filtered = [(ix, i) for ix, i in enumerate(List) if item in str(i)]

    if match_stem and len(filtered) > 1:
        filtered = [(ix, f) for ix, f in filtered if stem(f) == item]

    if len(filtered) == 1:
        if return_indexes_also:
            return filtered[0]
        else:
            return filtered[0][1]
    elif len(filtered) == 0:
        return None
    else:
        ixs, filtered_items = list(zip(*filtered))
        if return_indexes_also:
            return ixs, filtered_items
        else:
            return filtered_items

In [17]:
p = P("test.txt")
p.touch()
assert isdir(p) == False
assert fname(p) == "test.txt"
assert parent(p) == P()
assert stem(p) == "test"
assert extn(p) == "txt"

print(find("capsule", Glob("./")))

p.rm(confirm_prompt=False)

capsule.ipynb


In [18]:
# | export
import zipfile
import tarfile


def zip_files(list_of_files, dest):
    dest = str(dest)
    logger.info(f"Zipping {len(list_of_files)} files to {dest}...")
    if dest.lower().endswith(".zip"):
        with zipfile.ZipFile(dest, "w") as zipMe:
            for file in Tqdm(list_of_files):
                zipMe.write(file, compress_type=zipfile.ZIP_DEFLATED)
    elif dest.lower().endswith(".tar.gz"):
        with tarfile.open(dest, "w:gz") as tarMe:
            for file in Tqdm(list_of_files):
                tarMe.add(file)
    return P(dest)


def unzip_file(file, dest):
    file = str(file)
    if file.lower().endswith(".zip"):
        with zipfile.ZipFile(file, "r") as zip_ref:
            zip_ref.extractall(dest)
    elif file.lower().endswith(".tar.xz") or file.endswith(".tar.gz"):
        with tarfile.open(file, "r") as f:
            f.extractall(dest)
    return P(dest)


def list_zip(file):
    elements = []
    with zipfile.ZipFile(file, "r") as zipObj:
        listOfiles = zipObj.namelist()
        for elem in listOfiles:
            elements.append(elem)
    return elements

In [19]:
!touch test1.txt test2.txt
f = zip_files(P().Glob("*.txt"), "test.tar.gz")
unzip_file(f, "./")
[f.rm(force=True) for f in P().Glob("*.txt")]
P("test.tar.gz").rm(force=True)

!touch test1.txt test2.txt
f = zip_files(P().Glob("*.txt"), "test.zip")
unzip_file(f, "./")
[f.rm(force=True) for f in P().Glob("*.txt")]
P("test.zip").rm(force=True)

                    INFO     Zipping 2 files to test.tar.gz...                                                                                     ]8;id=907379;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/3826617683.py:8\3826617683.py]8;;\:]8;id=951098;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/3826617683.py:8#zip_files:8\zip_files:8]8;;\

100%|██████████| 2/2 [00:00<00:00, 649.78it/s]
/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/3826617683.py:27: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  f.extractall(dest)


                    INFO     Zipping 2 files to test.zip...                                                                                        ]8;id=235490;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/3826617683.py:8\3826617683.py]8;;\:]8;id=29273;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/3826617683.py:8#zip_files:8\zip_files:8]8;;\

100%|██████████| 2/2 [00:00<00:00, 2958.94it/s]


In [20]:
# | export
def md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


def remove_duplicates(files):
    """Check a list of files and remove duplicates based on their checksum"""
    import pandas as pd
    from torch_snippets.loader import diff

    hashes = [md5(f) for f in files]
    df = pd.DataFrame({"f": files, "h": hashes})
    x = df.drop_duplicates("h")
    y = diff(files, x.f)
    for i in y:
        os.rename(i, "./x")
    # !rm ./x
    return


def common_items(*fldrs, verbose=True):
    fldr_items = [stems(fldr) for fldr in fldrs]
    o = sorted(common(*fldr_items))
    if verbose:
        Info(
            f"Returning {len(o)} common items from folders of {[len(_f) for _f in fldr_items]} items each"
        )
    return o


def folder_summary(thing):
    things = Glob(thing)
    info = []
    for thing in things:
        if thing.is_dir():
            info.append(f"{thing} - {len(Glob(thing))} items")
        else:
            info.append(f"{thing} - {thing.size()}")
    return "\n".join(info)


print_folder_summary = lambda x: print(folder_summary(x))

In [21]:
md5("paths.ipynb")

'74eb392fa44e9293fb30ffed68c14112'

In [ ]:
# | export
def readlines(fpath, silent=False, encoding=None, _strip=True):
    with open(fpath, "r", encoding=encoding) as f:
        lines = f.read().split("\n")
        if _strip:
            lines = [l.strip() for l in lines if l.strip() != ""]
        if not silent:
            logger.opt(depth=1).log("INFO", f"loaded {len(lines)} lines")
        return lines


@patch_to(P)
def read_lines(self, silent=False, encoding=None):
    return readlines(self, silent=silent, encoding=encoding)


def readfile(*args, **kwargs):
    kwargs.pop("_strip", None)
    return "\n".join(readlines(*args, _strip=False, **kwargs)).strip()


@patch_to(P)
def read_file(self, **kwargs):
    return readfile(self, **kwargs)


def printfile(*args, **kwargs):
    print(readfile(*args, **kwargs))


@patch_to(P)
def print_file(self, **kwargs):
    return print(self.read_file(**kwargs))


def writelines(lines, file, mode):
    makedir(parent(file))
    failed = []
    with open(file, mode) as f:
        for line in lines:
            try:
                f.write(f"{line}\n")
            except:
                failed.append(line)
    if failed != []:
        logger.opt(depth=1).log(
            "INFO", f"Failed to write {len(failed)} lines out of {len(lines)}"
        )
        return failed


@patch_to(P)
def write_lines(self, lines, mode):
    return writelines(lines, self, mode)

In [23]:
P("paths.ipynb").read_lines()[:10]

                    INFO     loaded 1075 lines                                                                                                   ]8;id=963649;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/2901855681.py:13\2901855681.py]8;;\:]8;id=662284;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/2901855681.py:13#read_lines:13\read_lines:13]8;;\

['{',
 '"cells": [',
 '{',
 '"cell_type": "raw",',
 '"metadata": {},',
 '"source": [',
 '"# Paths\\n",',
 '"Utilities to manipulate Paths\\n",',
 '"---\\n",',
 '"{}\\n",']

In [24]:
# | export
def tree(directory="./", filelimit=50, to=None):
    from builtins import print

    # Construct the shell command
    directory = str(P(directory).resolve())
    shell_command = f"tree \"{directory}\" --filelimit={filelimit} | sed 's/├/ /g; s/└/ /g; s/|/ /g; s/`/ /g; s/│/ /g; s/+/ /g'"
    # Execute the shell command
    try:
        result = subprocess.run(
            shell_command, shell=True, capture_output=True, text=True
        )
        # Print the output
        if to:
            writelines(result.stdout.split("\n"), to, mode="w")
        else:
            print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error executing command: {e}")


@patch_to(P, as_prop=True)
def _tree(self, filelimit=50, to=None):
    return tree(self, filelimit, to)


P.tree = P._tree

In [25]:
P("../").tree

/Users/yeshwanth/Code/Personal/torch_snippets
 ── LICENSE
 ── LICENSE.txt
 ── MANIFEST
 ── MANIFEST.in
 ── Makefile
 ── README.md
 ── Screenshot 2022-09-08 at 4.40.18 PM.png
 ── __module_timing__.py.lprof
 ── _proc
     ── AttrDict.ipynb
     ── _quarto.yml
     ── adapters.ipynb
     ── bokeh_plotting.ipynb
     ── bounding_boxes.ipynb
     ── capsule.ipynb
     ── charts.ipynb
     ── config.ipynb
     ── decorators.ipynb
     ── docs
         ── AttrDict.html
         ── adapters.html
         ── bokeh_plotting.html
         ── bounding_boxes.html
         ── bounding_boxes_files
             ── figure-html
                 ── cell-6-output-1.png
                 ── cell-7-output-1.png
                 ── cell-8-output-1.png
                 ── cell-9-output-1.png
         ── capsule.html
         ── capsule_files
             ── figure-html
                 ── cell-6-output-2.png
         ── charts.html
         ── charts_files
             ── figure-html
                 ── cell-7

In [26]:
# | export
def folder_structure_to_dict(path):
    """
    Recursively constructs a nested dictionary that represents the folder structure.
    """
    folder_dict = {}
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_dir():
                folder_dict[entry.name] = folder_structure_to_dict(entry.path)
            else:
                folder_dict[entry.name] = None
    return folder_dict


def folder_structure_to_json(path, output_file=None):
    """
    Creates a JSON file representing the folder structure of the given directory.
    """
    path = P(path)
    if output_file is None:
        output_file = path / "tree.json"
    folder_dict = folder_structure_to_dict(path)
    with open(output_file, "w") as f:
        json.dump(folder_dict, f, indent=4)

In [27]:
x = P("tmp.txt")
x.touch()
x.write_lines([i for i in range(1000)], mode="w")
lines = x.read_lines()
assert lines == [f"{i}" for i in range(1000)]
logger.info(x.size())

x.rm(confirm_prompt=False)

[08/24/24 11:12:13] INFO     loaded 1000 lines                                                                                                   ]8;id=250917;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/2901855681.py:13\2901855681.py]8;;\:]8;id=777128;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/2901855681.py:13#read_lines:13\read_lines:13]8;;\

                    INFO     3 KB                                                                                                                   ]8;id=311863;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/1033976698.py:6\1033976698.py]8;;\:]8;id=855849;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/1033976698.py:6#<module>:6\<module>:6]8;;\

In [28]:
# | export
# | hide
def rename_batch(folder, func, debug=False, one_file=False):
    from torch_snippets.loader import now

    "V.V.Imp: Use debug=True first to confirm file name changes are as expected"
    if isinstance(folder, (str, P)):
        folder = Glob(folder)
    sources = []
    destins = []
    log_file = f"moved_files_{now()}.log"
    for f in folder:
        source = f
        destin = func(f)
        if source == destin:
            continue
        if debug:
            logger.debug(f"moving `{source}` --> `{destin}`")
        else:
            # !mv {source.replace(' ','\ ')} {destin.replace(' ','\ ')}
            logger.info(f"moving `{source}` --> `{destin}`")
            os.rename(source, destin)
        # !echo {source.replace(' ','\ ')} --\> {destin.replace(' ','\ ')} >> {logfile}
        if one_file:
            break

In [29]:
# | export
dill = dill


def dumpdill(
    obj,
    fpath,
    silent=False,
    message='Dumped object of size ≈{fsize} @ "{fpath}" in {dumptime:.2e} seconds',
):
    """Dump a python object as a dill file (better replacement to pickle)"""
    start = time.time()
    fpath = P(fpath)
    fpath.parent.mkdir(exist_ok=True, parents=True)
    with open(fpath, "wb") as f:
        dill.dump(obj, f)
    dumptime = time.time() - start
    if not silent:
        fsize = fpath.size()
        logger.opt(depth=1).log(
            "INFO", message.format(fsize=fsize, fpath=fpath, dumptime=dumptime)
        )
    return P(fpath)


def loaddill(fpath):
    """Load a python object from a dill file"""
    fpath = str(fpath)
    with open(fpath, "rb") as f:
        obj = dill.load(f)
    return obj

In [30]:
p = P("test.tmp")
dumpdill([1, 2, 3], p)
y = loaddill(p)
p.rm(confirm_prompt=False)
assert y == [1, 2, 3]

                    INFO     Dumped object of size ≈0 KB @ "test.tmp" in 4.11e-04 seconds                                                            ]8;id=187347;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/554288780.py:2\554288780.py]8;;\:]8;id=958846;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_40147/554288780.py:2#<module>:2\<module>:2]8;;\